In [5]:
import pandas as pd
import numpy as np


# Carregar os dados
df = pd.read_csv('Dataset/train.csv')
df = df.drop(columns=['AnimalID','Name','OutcomeSubtype'])
df = df.dropna()

# Exibir os dados
display(df)

,DateTime,OutcomeType,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
0,2014-02-12 18:22:00,Return_to_owner,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
1,2013-10-13 12:44:00,Euthanasia,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
2,2015-01-31 12:28:00,Adoption,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White
3,2014-07-11 19:09:00,Transfer,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream
4,2013-11-15 12:52:00,Transfer,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan
...,...,...,...,...,...,...,...
26724,2015-05-14 11:56:00,Transfer,Cat,Intact Male,1 month,Domestic Shorthair Mix,Brown Tabby/White
26725,2016-01-20 18:59:00,Transfer,Cat,Spayed Female,3 months,Domestic Shorthair Mix,Brown Tabby
26726,2015-03-09 13:33:00,Adoption,Dog,Neutered Male,4 years,Old English Bulldog Mix,White/Tan
26727,2014-04-27 12:22:00,Transfer,Cat,Intact Male,4 weeks,Domestic Shorthair Mix,Black


In [7]:
# Função para converter a idade (anos/meses) em semanas
def age_to_weeks(age):
    if 'year' in age:
        years = int(age.split()[0]) # Captura o valor númerico antes da substring "year"
        return years * 52 # Converte o ano para semanas (1 ano == 52 semanas)
    elif 'month' in age:
        months = int(age.split()[0]) # Captura o valor númerico antes da substring "month"
        return months * 4 # Converte o mês para semanas (1 mês = 4 semanas)
    else:
        return None # Caso haja um valor fora dos parâmetros previstos

# Dicionário de mapeamento de cores
color_map = {
    'laranja': ['orange', 'red', 'tan', 'buff', 'flame'],
    'preto': ['black', 'seal', 'brown brindle', 'torbie', 'tortie', 'tricolor', 'calico'],
    'branco': ['white', 'cream'],
    'marrom': ['brown', 'chocolate', 'fawn'],
    'cinza': ['gray', 'blue', 'silver'],
    'amarelo': ['yellow'],
}

# Função para mapear uma cor específica para uma cor primária
def map_color(color):
    color = color.lower()  # Convertendo para minúsculas para facilitar a comparação
    for primary_color, color_list in color_map.items():
        for specific_color in color_list:
            if specific_color in color:
                return primary_color
    return color  # Se a cor não estiver no mapeamento, mantemos como está

# Função para tratar múltiplas cores (transição de cores)
def handle_multiple_colors(color):
    # Separar cores que estão com '/' ou ' '
    if '/' in color:
        colors = color.split('/')
    elif ' ' in color:
        colors = color.split(' ')
    else:
        colors = [color]

    # Mapear cada cor separadamente
    mapped_colors = [map_color(c) for c in colors]

    # Retornar as cores combinadas (ex: 'laranja/branco')
    return '/'.join(mapped_colors)

# Aplicar o tratamento para múltiplas cores na coluna 'Color'
df['Mapped_Color'] = df['Color'].apply(handle_multiple_colors)

# Exibir as primeiras linhas com as cores mapeadas
df[['Color', 'Mapped_Color']].head(20)


,Color,Mapped_Color
0,Brown/White,marrom/branco
1,Cream Tabby,branco/tabby
2,Blue/White,cinza/branco
3,Blue Cream,cinza/branco
4,Tan,laranja
5,Black/Tan,preto/laranja
6,Blue Tabby,cinza/tabby
7,Brown Tabby,marrom/tabby
8,Red/White,laranja/branco
9,White,branco


In [8]:
# 1) Análise de proporção da coluna OutcomeType
outcome_proportion = df['OutcomeType'].value_counts(normalize=True)*100 # Normalize retorna a proporção de cada elemento == Nro_elementoX/Nro_total
print("Proporção de cada item do OutcomeType:")
print(outcome_proportion)

# 2) Análise de proporção da coluna AnimalType
animal_proportion = df['AnimalType'].value_counts(normalize=True)*100
print("\nProporção de cada item do AnimalType:")
print(animal_proportion)

# 3) Análise de proporção da coluna Breed (Mestiça X Pura)
# Criação de uma subtabela para classificação de raça mestiça ou pura
df['BreedType'] = df['Breed'].apply(lambda x: 'Mestiça' if 'Mix' in x else 'Pura') #lambda x é uma função temporária que tem como entrada o parâmetro x (elemento da coluna raça)
                                                                                   # Neste caso, se na string tiver 'Mix' como substring classifica como mestiça, se não pura
breed_proportion = df['BreedType'].value_counts(normalize=True)*100
print("\nProporção de BreedType (Mestiça/Pura)")
print(breed_proportion)

# 4) Análise de proporção da coluna AgeuponOutcome
df['AgeuponOutcomeWeeks'] = df['AgeuponOutcome'].apply(age_to_weeks) # Conversão de meses e anos para semanas

# Definir os intervalos discretos (bins) de 52 em 52 semanas, de 0 até 1040
bins = np.arange(0, 1041, 52)  # Isso cria intervalos de 52 semanas (O que é por volta de 1 ano)

# Criar uma coluna que indique em qual intervalo cada valor de 'Age_in_Weeks' cai
df['Intervalo'] = pd.cut(df['AgeuponOutcomeWeeks'], bins=bins, right=False)

frequencia_absoluta = df['Intervalo'].value_counts().sort_index() # Calcular a frequência absoluta (número de observações em cada intervalo)
frequencia_relativa = frequencia_absoluta / len(df) * 100 # Calcular a frequência relativa (proporção de observações em cada intervalo)

# Criar uma tabela com as frequências absoluta e relativa
tabela_frequencia = pd.DataFrame({
    'Frequência Absoluta': frequencia_absoluta,
    'Frequência Relativa (%)': frequencia_relativa
})

print("\nProporção de AgeuponOutcome (em semanas)")
print(tabela_frequencia)

# 5) Análise de proporção da coluna SexuponOutcome
# A) Proporção de castrados e não castrados
castrated_proportion = df['SexuponOutcome'].apply(lambda x: 'Castrado' if 'Spayed' in x or 'Neutered' in x else 'Não Castrado').value_counts(normalize=True) * 100
print("\nProporção Castrados e Não Castrados:")
print(castrated_proportion)

# B) Proporção de Fêmeas e Machos
gender_proportion = df['SexuponOutcome'].apply(lambda x: 'Macho' if 'Male' in x else 'Fêmea').value_counts(normalize=True) * 100
print("\nProporção de Machos e Fêmeas:")
print(gender_proportion)

# C) Proporção de Fêmeas e Machos castrados e não castrados
gender_castrated_proportion = df['SexuponOutcome'].apply(lambda x:
    'Macho Castrado' if 'Neutered Male' in x else
    'Macho Não Castrado' if 'Intact Male' in x else
    'Fêmea Castrada' if 'Spayed Female' in x else
    'Fêmea Não Castrada').value_counts(normalize=True) * 100
print("\nProporção de Machos e Fêmeas (Castrados e Não Castrados):")
print(gender_castrated_proportion)

# Filtrar apenas Fêmeas (Spayed Female e Intact Female)
females_only = df['SexuponOutcome'].apply(lambda x: 'Fêmea Castrada' if 'Spayed Female' in x
                                          else 'Fêmea Não Castrada' if 'Intact Female' in x else None)

# Remover valores None (caso sejam machos ou valores desconhecidos)
females_only = females_only.dropna()

# Calcular a proporção de Fêmeas castradas e não castradas
gender_castrated_proportion = females_only.value_counts(normalize=True) * 100
print("\nProporção de Fêmeas (Castradas e Não Castradas):")
print(gender_castrated_proportion)


# Filtrar apenas Machos (Neutered Male e Intact Male)
males_only = df['SexuponOutcome'].apply(lambda x: 'Macho Castrado' if 'Neutered Male' in x
                                        else 'Macho Não Castrado' if 'Intact Male' in x else None)

# Remover valores None (caso sejam fêmeas ou valores desconhecidos)
males_only = males_only.dropna()

# Calcular a proporção de Machos castrados e não castrados
gender_castrated_proportion = males_only.value_counts(normalize=True) * 100
print("\nProporção de Machos (Castrados e Não Castrados):")
print(gender_castrated_proportion)



# 6) Análise de proporção da coluna color
color_proportion = df['Color'].value_counts(normalize=True)*100
print("\nProporção de Color:")
print(len(color_proportion))

def padronizar_cores(cor):
    cores = cor.split('/')  # Divide a string em partes
    cores.sort()  # Ordena as cores em ordem alfabética
    return '/'.join(cores)  # Junta as cores novamente

df['Color'] = df['Color'].apply(padronizar_cores)

color_proportion = df['Color'].value_counts(normalize=True)*100
print("\nProporção de Color:")
print(len(color_proportion))

for row in range(len(color_proportion)):
  indice = color_proportion.index[row]
  valor = color_proportion.iloc[row]
  print(f'Índice: {indice}, Valor: {valor}')

Proporção de cada item do OutcomeType:
OutcomeType
Adoption           40.318233
Transfer           35.215275
Return_to_owner    17.914639
Euthanasia          5.814302
Died                0.737551
Name: proportion, dtype: float64

Proporção de cada item do AnimalType:
AnimalType
Dog    58.378884
Cat    41.621116
Name: proportion, dtype: float64

Proporção de BreedType (Mestiça/Pura)
BreedType
Mestiça    83.414452
Pura       16.585548
Name: proportion, dtype: float64

Proporção de AgeuponOutcome (em semanas)
             Frequência Absoluta  Frequência Relativa (%)
Intervalo                                                
[0, 52)                     9642                36.098839
[52, 104)                   3969                14.859603
[104, 156)                  3742                14.009734
[156, 208)                  1823                 6.825159
[208, 260)                  1071                 4.009734
[260, 312)                   992                 3.713965
[312, 364)              